In [1]:
pip install feature-engine

     |████████████████████████████████| 205 kB 13.9 MB/s            
     |████████████████████████████████| 9.5 MB 28 kB/s              
     |████████████████████████████████| 233 kB 31 kB/s              
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

In [3]:
data=pd.read_csv("heart.csv")

In [4]:
data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
data.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

# Train Test Split 

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
drop = ['oldpeak','slp','thall']
data = data.drop(drop, axis=1)

In [8]:
data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,caa,output
0,63,1,3,145,233,1,0,150,0,0,1
1,37,1,2,130,250,0,1,187,0,0,1
2,41,0,1,130,204,0,0,172,0,0,1
3,56,1,1,120,236,0,1,178,0,0,1
4,57,0,0,120,354,0,1,163,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0,0
299,45,1,3,110,264,0,1,132,0,0,0
300,68,1,0,144,193,1,1,141,0,2,0
301,57,1,0,130,131,0,1,115,1,1,0


In [9]:
Y = data['output']
X = data.drop('output',axis = 1)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1) 

# Standardization & Feature Selection

In [11]:
from sklearn.pipeline import Pipeline
from feature_engine.selection import DropConstantFeatures, DropDuplicateFeatures, SmartCorrelatedSelection

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=10, random_state=20)

In [13]:
pipe=Pipeline([('constant', DropConstantFeatures(tol=0.997)),('Duplicate', DropDuplicateFeatures()),('correlated', SmartCorrelatedSelection(selection_method="model_performance", estimator=rf, scoring="roc_auc", cv=3))])

In [14]:
pipe.fit(X_train, Y_train)

Pipeline(steps=[('constant', DropConstantFeatures(tol=0.997)),
                ('Duplicate', DropDuplicateFeatures()),
                ('correlated',
                 SmartCorrelatedSelection(estimator=RandomForestClassifier(n_estimators=10,
                                                                           random_state=20),
                                          selection_method='model_performance'))])

In [15]:
pipe.named_steps['correlated'].features_to_drop_

[]

In [16]:
X_train=pipe.transform(X_train)
X_test=pipe.transform(X_test)

In [17]:
X_train.shape

(242, 10)

In [18]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [19]:
from sklearn.preprocessing import StandardScaler
scal=StandardScaler()
scal.fit(X_train)

StandardScaler()

In [20]:
sel=SelectFromModel(LogisticRegression(C=0.8, penalty='l1', solver='liblinear', random_state=10))

In [21]:
sel.fit_transform(scal.transform(X_train), Y_train)

array([[-0.27090572,  0.6636838 ,  1.9766492 , ...,  1.72150023,
        -0.68313005, -0.72428597],
       [ 1.3708101 , -1.50674161,  0.99843017, ...,  0.95592034,
        -0.68313005,  0.27160724],
       [ 0.27633288,  0.6636838 ,  0.99843017, ...,  0.99845256,
        -0.68313005,  0.27160724],
       ...,
       [-2.78820331,  0.6636838 ,  0.02021114, ...,  2.23188682,
        -0.68313005, -0.72428597],
       [-0.38035344,  0.6636838 , -0.95800789, ...,  0.99845256,
         1.46385011, -0.72428597],
       [-0.05201028,  0.6636838 ,  0.99843017, ...,  0.65819483,
        -0.68313005, -0.72428597]])

In [22]:
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [23]:
sel_features = X_train.columns[sel.get_support()]
len(sel_features)

10

# Logistic Regression

In [24]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(X_train,Y_train)

/srv/conda/envs/notebook/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

# y_pred=reg.predict(X_test)
y_pred

In [26]:
reg.score(X_test,Y_test)

0.6885245901639344

# Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
regdT=DecisionTreeClassifier(criterion='gini')
regdT.fit(X_train,Y_train)

DecisionTreeClassifier()

In [29]:
regdT.predict(X_test)

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0])

In [31]:
regdT.score(X_test,Y_test)

0.6721311475409836

# Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
regrFo=RandomForestClassifier(n_estimators=180)
regrFo.fit(X_train,Y_train)

RandomForestClassifier(n_estimators=180)

In [34]:
regrFo.predict(X_test)

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0])

In [35]:
regrFo.score(X_test,Y_test)

0.6885245901639344

# KNN

In [36]:
from sklearn.neighbors import KNeighborsClassifier

In [37]:
KNN=KNeighborsClassifier(n_neighbors=14)
KNN.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=14)

In [38]:
KNN.predict(X_test)

array([1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0])

In [39]:
KNN.score(X_test,Y_test)

0.5737704918032787

# SVM

In [40]:
from sklearn.svm import SVC


In [41]:
svc = SVC(kernel='linear')
svc.fit(X_train, Y_train)
svc.predict(X_test)

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0])

In [42]:
svc.score(X_test,Y_test)

0.7049180327868853